In [1]:
TODO: Change hyperparams, show overall comparisions amongst all the three models

Also show tensorboard improvements per model from basic to this improved on

## Optional

* Try various data augmentation strategies to further improve your accuracy.
* Add Tensorboard graphs in your notebook. `Done`

SyntaxError: invalid syntax (2669961143.py, line 1)

In [ ]:
Keep in mind that the following are also available:

experiment with the optimizer: type of optimizer, learning rate, scheduler etc
experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.